In [1]:
import pdfplumber
import pandas as pd

In [2]:
PDF_PATH = "DP.pdf"

In [3]:
tables = []
with pdfplumber.open(PDF_PATH) as pdf:
        for page in pdf.pages:
            t = page.extract_tables()
            if t:
                tables.extend(t)    
len(tables)                

172

In [4]:
final_table = []
for table in tables:
    for row in table:
        if row[1] == "bakalavriat" or row[1] == "magistratura":
            final_table.append(row)

df = pd.DataFrame(final_table, columns=['Nömrə', 'Təhsil səviyyəsi', 'Ölkə', 'Universitet', 'Təhsil proqramı'])
df.head()

,Nömrə,Təhsil səviyyəsi,Ölkə,Universitet,Təhsil proqramı
0,1,bakalavriat,Almaniya Federativ Respublikası,"""Technical University of Munich""","""Architecture"""
1,2,bakalavriat,Almaniya Federativ Respublikası,"""Technical University of Munich""","""Landscape Architecture and Landscape Planning"""
2,3,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""Abu Dhabi University""","""Architecture"""
3,4,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""Abu Dhabi University""","""Interior Design"""
4,5,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""United Arab Emirates University""","""Architectural Engineering"""


In [5]:
df.shape

(4405, 5)

In [6]:
# let's rename the columns to English and add ID column instead of the first column
df.columns = ['id', 'education_level', 'country', 'university', 'program']
df['id'] = range(1, len(df) + 1)
df.head()

,id,education_level,country,university,program
0,1,bakalavriat,Almaniya Federativ Respublikası,"""Technical University of Munich""","""Architecture"""
1,2,bakalavriat,Almaniya Federativ Respublikası,"""Technical University of Munich""","""Landscape Architecture and Landscape Planning"""
2,3,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""Abu Dhabi University""","""Architecture"""
3,4,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""Abu Dhabi University""","""Interior Design"""
4,5,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""United Arab Emirates University""","""Architectural Engineering"""


In [9]:
# "TUM" -> TUM
df['university'] = df['university'].str.replace('"', '')
df['program'] = df['program'].str.replace('"', '')
df.head()

,id,education_level,country,university,program
0,1,bakalavriat,Almaniya Federativ Respublikası,Technical University of Munich,Architecture
1,2,bakalavriat,Almaniya Federativ Respublikası,Technical University of Munich,Landscape Architecture and Landscape Planning
2,3,bakalavriat,Birləşmiş Ərəb Əmirlikləri,Abu Dhabi University,Architecture
3,4,bakalavriat,Birləşmiş Ərəb Əmirlikləri,Abu Dhabi University,Interior Design
4,5,bakalavriat,Birləşmiş Ərəb Əmirlikləri,United Arab Emirates University,Architectural Engineering


In [10]:
df.to_csv("DP.csv", index=False)

In [22]:
rankings = pd.read_csv("qs-world-rankings-2025.csv")
# trim institution names
rankings['Institution Name'] = rankings['Institution Name'].str.strip()
rankings.head()

,2025 Rank,2024 Rank,Institution Name,Location,Location Full,Size,Academic Reputation,Employer Reputation,Faculty Student,Citations per Faculty,International Faculty,International Students,International Research Network,Employment Outcomes,Sustainability,QS Overall Score
0,1,1,Massachusetts Institute of Technology (MIT),US,United States,M,100.0,100.0,100.0,100.0,99.3,86.8,96.0,100.0,99.0,100
1,2,6,Imperial College London,UK,United Kingdom,L,98.5,99.5,98.2,93.9,100.0,99.6,97.4,93.4,99.7,98.5
2,3,3,University of Oxford,UK,United Kingdom,L,100.0,100.0,100.0,84.8,98.1,97.7,100.0,100.0,85.0,96.9
3,4,4,Harvard University,US,United States,L,100.0,100.0,96.3,100.0,74.1,69.0,99.6,100.0,84.4,96.8
4,5,2,University of Cambridge,UK,United Kingdom,L,100.0,100.0,100.0,84.6,100.0,94.8,99.3,100.0,84.8,96.7


In [36]:
for idx, row in df.iterrows():
    for _, ranking_row in rankings.iterrows():
        # Use .lower() on strings
        if row['university'].lower() in ranking_row['Institution Name'].lower():
            df.loc[idx, 'ranking'] = ranking_row['2025 Rank']
            df.loc[idx, 'academic_reputation'] = ranking_row['Academic Reputation']
            df.loc[idx, 'employer_reputation'] = ranking_row['Employer Reputation']
            df.loc[idx, 'faculty_student'] = ranking_row['Faculty Student']
            df.loc[idx, 'citations_per_faculty'] = ranking_row['Citations per Faculty']
            df.loc[idx, 'international_faculty'] = ranking_row['International Faculty']
            df.loc[idx, 'international_students'] = ranking_row['International Students']
            df.loc[idx, 'international_research_network'] = ranking_row['International Research Network']
            df.loc[idx, 'employment_outcomes'] = ranking_row['Employment Outcomes']
            df.loc[idx, 'sustainability'] = ranking_row['Sustainability']
            df.loc[idx, 'qs_overall_score'] = ranking_row['QS Overall Score']
            break


In [37]:
df[df['ranking'].isnull()].university.unique().shape

(51,)

In [38]:
df.head()

,id,education_level,country,university,program,ranking,academic_reputation,employer_reputation,faculty_student,citations_per_faculty,international_faculty,international_students,international_research_network,employment_outcomes,sustainability,qs_overall_score
0,1,bakalavriat,Almaniya Federativ Respublikası,Technical University of Munich,Architecture,28,83.0,98.6,76.8,75.9,80.4,98.6,95.6,34.5,93.1,83.2
1,2,bakalavriat,Almaniya Federativ Respublikası,Technical University of Munich,Landscape Architecture and Landscape Planning,28,83.0,98.6,76.8,75.9,80.4,98.6,95.6,34.5,93.1,83.2
2,3,bakalavriat,Birləşmiş Ərəb Əmirlikləri,Abu Dhabi University,Architecture,501,15.9,24.6,14.9,16.7,100.0,99.8,27.3,8.3,1.5,24.1
3,4,bakalavriat,Birləşmiş Ərəb Əmirlikləri,Abu Dhabi University,Interior Design,501,15.9,24.6,14.9,16.7,100.0,99.8,27.3,8.3,1.5,24.1
4,5,bakalavriat,Birləşmiş Ərəb Əmirlikləri,United Arab Emirates University,Architectural Engineering,261,30.1,38.3,49.8,16.6,100.0,34.2,86.2,82.0,36.6,39.4


In [39]:
df.to_csv("DP.csv", index=False)